In [ ]:
# --- Install dependencies ---
!pip install -q transformers datasets peft bitsandbytes accelerate

In [ ]:
# --- Imports ---
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch

In [ ]:
# --- Load base model and tokenizer ---
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = 'EleutherAI/gpt-neo-125M'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# Fix padding issue for GPT-style models
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto')


In [ ]:
# --- Load dataset ---
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train[:1%]')  # small subset for testing

In [ ]:
# --- Tokenization ---
def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        truncation=True, 
        padding='max_length',  # now safe, because pad_token is set
        max_length=128
    )

tokenized = dataset.map(tokenize_function, batched=True)
tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])


In [ ]:
# --- Configure LoRA ---
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, lora_config)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="role-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to=[]  # <<< disables wandb/other loggers
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized
)

trainer.train()


In [ ]:
# --- Save model ---
model.save_pretrained('role-lora-final')
tokenizer.save_pretrained('role-lora-final')